## Creating Tweet Dataset

In [1]:
import GetOldTweets3 as got

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

#from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

import string

from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist

import re

/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
pd.set_option('display.max_rows', 100)
pd.options.display.max_colwidth = 150

#### Retrieving Data from Twitter API via GetOldTweets3

In [6]:
tweetCriteria = got.manager.TweetCriteria().setQuerySearch('anxiety')\
                                           .setSince('2018-06-13')\
                                           .setUntil('2020-06-13')\
                                           .setNear('New York')\
                                           .setMaxTweets(12000)\
                                           .setEmoji('unicode')
                                           

tweet_object = got.manager.TweetManager.getTweets(tweetCriteria)

In [95]:
len(tweet_object)

12000

In [13]:
#construct a dictionary of desired values

tweet_dict = list(map(lambda x: {'text': x.text, 'user': x.username,
                                 'date': x.date, 'retweet': x.retweets,
                                 'mention': x.mentions, 'hashtags': x.hashtags,
                                 'location' : x.geo}, tweet_object))


In [17]:
#preview dictionary
tweet_dict[0]

{'text': 'Had anxiety the whole time I’m up here cause I dont do heights but was worth the view 🥴 ',
 'user': 'MxhmoudG',
 'date': datetime.datetime(2020, 6, 12, 23, 53, 55, tzinfo=datetime.timezone.utc),
 'retweet': 0,
 'mention': '',
 'hashtags': '',
 'location': ''}

In [18]:
#turn dictionary into dataframe 

df = pd.DataFrame(tweet_dict)

In [19]:
#preview dataframe
print(df.shape)
df.head()


(12000, 7)


,text,user,date,retweet,mention,hashtags,location
0,Had anxiety the whole time I’m up here cause I dont do heights but was worth the view 🥴,MxhmoudG,2020-06-12 23:53:55+00:00,0,,,
1,#coconutoil🥥 is a great holistic treatment for #memoryloss #anxiety #Depresion #alzheimers and any other #brain🧠 related condition 🤓 . . #MyelinSh...,Jahypimentel,2020-06-12 23:50:32+00:00,0,,#coconutoil #memoryloss #anxiety #Depresion #alzheimers #brain #MyelinSheath,
2,"Will be so glad to not feel constantly bogged down from anxiety related to loneliness, hopelessness, &amp; misanthropy. Also my flowers look rly c...",RaverHippie,2020-06-12 23:48:46+00:00,0,,,
3,English,Clarknt67,2020-06-12 22:58:39+00:00,0,,,
4,"Tbh, since ive been taking time to meditate, my anxiety hasnt really been acting up and i love it",chiqui_xo,2020-06-12 22:58:09+00:00,0,,,


In [75]:
#write tweets to csv 
df.to_csv('csv_files/anxiety_tweets.csv', index = False)

## Building New Tweet Set with Nothing specified in Query Search

In [2]:
tweetCriteria = got.manager.TweetCriteria().setQuerySearch('')\
                                           .setSince('2018-06-30')\
                                           .setUntil('2020-06-30')\
                                            .setNear('New York')\
                                           .setMaxTweets(12000)
                                           
                                           

tweet_object = got.manager.TweetManager.getTweets(tweetCriteria)

In [20]:
#function for turning tweet return object to df

def tweet_to_df(tweet_object):
    
    
    tweet_dict = list(map(lambda x: {'text': x.text, 'user': x.username,
                                     'date': x.date, 'retweet': x.retweets,
                                     'mention': x.mentions, 'hashtags': x.hashtags,
                                     'location' : x.geo}, tweet_object))
    
    tweet_df = pd.DataFrame(tweet_dict)
    
    return tweet_df

df = tweet_to_df(tweet_object)

In [22]:
df.shape

(12000, 7)

In [ ]:
#save to csv
df.to_csv('csv_files/random_tweets.csv', index = False)